In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from IPython.display import display

In [23]:
n = 100 # time points to plot
tf = 500# final time
SP_start = 25 # time of set point change

def process(y,t,u):
    K = 4.0
    taup = 3.0
    thetap = 1.0
    if t<(thetap+SP_start):
        dydt = 0.0  # time delay
    else:
        dydt = (1.0/taup) * (-y + K * u)
    return dydt

In [24]:
# Algorithm of PID
def pidPlot(kp,ki,kd):
    t = np.linspace(0,tf,n) # create time vector
    P= np.zeros(n)          # initialize proportional term
    I = np.zeros(n)         # initialize integral term
    D = np.zeros(n)         # initialize derivative term
    e = np.zeros(n)         # initialize error
    OP = np.zeros(n)        # initialize controller output
    PV = np.zeros(n)        # initialize process variable
    
    
    
    
#   Setting a setfunction to follow
    SP = np.zeros(n)        # initialize setpoint
    #SP_step = int(SP_start/(tf/(n-1))+1) # setpoint start
    SP[0:25] = 0.0     # define setpoint
    SP[25:60] = 4    # step up
    SP[60:n] = 2
    y0 = 0.0                # initial condition
    
    

#   loop through all time steps
    for i in range(1,n):
        # simulate process for one time step
        ts = [t[i-1],t[i]]         # time interval
        
#       Calling the process to calculate
        y = odeint(process,y0,ts,args=(OP[i-1],))  # compute next step
        y0 = y[1]                  # record new initial condition
#       calculate new OP with PID
        PV[i] = y[1]               # record PV
        e[i] = SP[i] - PV[i]       # calculate error = SP - PV
        dt = t[i] - t[i-1]         # calculate time step
        P[i] = kp * e[i]           # calculate proportional term
        I[i] = I[i-1] + ki * e[i] * dt  # calculate integral term
        D[i] = kd * (PV[i]-PV[i-1])/dt # calculate derivative term
        OP[i] = P[i] + I[i] + D[i] # calculate new controller output
        
#   Plot PID response
    plt.figure()

# Setpoint (SP)    &   Process Variable (PV)
    plt.plot(t,SP,'k',linewidth=2,label='Setpoint (SP)')
    plt.plot(t,PV,'r',linewidth=2,label='Process Variable (PV)')
    #plt.plot(t,OP,'b--',linewidth=2,label='Controller Output (OP)')
    plt.plot(t,e,'m--',linewidth=2,label='Error (e=SP-PV)')
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel('Temerature')
    plt.grid()
    plt.savefig("pid1.png")
    

    
    
# P,I and D component
    plt.figure()
    plt.plot(t,P,'r-',linewidth=2,label=r'Proportional = $K_c \; e(t)$')
    plt.plot(t,I,'g-',linewidth=2,label=r'Integral = $\frac{K_c}{\tau_I} \int_{i=0}^{n_t} e(t) \; dt $')
    plt.plot(t,D,'b-',linewidth=2,label=r'Derivative = $-K_c \tau_D \frac{d(PV)}{dt}$')   
    plt.xlabel('Time')
    plt.ylabel('Temerature')
    plt.legend(loc='best')
    plt.grid()
    plt.savefig("p_i_d.png")


In [25]:
import ipywidgets as wg    
Kp_slide = wg.FloatSlider(value=0.15,min=-0.2,max=1.0,step=0.05)
ki_slide = wg.FloatSlider(value=0.06,min=0.0,max=0.5,step=0.01)
kd_slide = wg.FloatSlider(value=0.18,min=0.0,max=1,step=0.01)
wg.interact(pidPlot, kp=Kp_slide, ki=ki_slide, kd=kd_slide)

interactive(children=(FloatSlider(value=0.15, description='kp', max=1.0, min=-0.2, step=0.05), FloatSlider(val…

<function __main__.pidPlot(kp, ki, kd)>